In [1]:
import pandas as pd
import numpy as np

In [2]:
mf = pd.read_csv("Master.csv")
bf = pd.read_csv("Batting.csv")
pf = pd.read_csv("Pitching.csv")

In [3]:
bf = bf[bf['yearID'] == 2015]
pf = pf[pf['yearID'] == 2015]

In [4]:
bf.columns

Index(['playerID', 'yearID', 'stint', 'teamID', 'lgID', 'G', 'AB', 'R', 'H',
       '2B', '3B', 'HR', 'RBI', 'SB', 'CS', 'BB', 'SO', 'IBB', 'HBP', 'SH',
       'SF', 'GIDP'],
      dtype='object')

In [5]:
#plate apperance
PA = bf['AB']+bf['BB']+bf['HBP']+bf['SF']
bf['PA'] = PA
#OBP
OBP = (bf['H']+bf['BB']+bf['HBP'])/bf['PA']
bf['OBP'] = OBP
#SLUGGING
#calculate singles
B = bf['H']-(bf['HR']+bf['3B']+bf['2B'])
bf['B'] = B
#slug
slug = (bf['B']+(2*bf['2B'])+(3*bf['3B'])+(4*bf['HR']))/bf['AB']
bf['SLUG'] = slug
#BA
ba = bf['H']/bf['AB']
bf['BA'] = ba
#drop all players with less than 50 plate appearance
bf = bf[bf['PA'] >=50]

In [6]:
#create a new dataframe to perform pca
bfn = bf[['playerID','yearID','teamID','PA','OBP','SLUG','BA','stint']]

In [7]:
import seaborn as sns

bcorr = bfn.corr()
sns.heatmap(bcorr)

In [8]:
#treating players with multiple stints 
#by averaging their their value
bfns = bfn.groupby(['playerID'], sort=True).mean().reset_index()

In [9]:
#drop the stint value to streamline the value 
#after sorting out the stint problem
bfns = bfns.drop(columns = 'stint')

In [10]:
from sklearn import preprocessing
from sklearn.decomposition import PCA 

X = bfns[['OBP','SLUG','BA']]

#scaling the data 
scal = preprocessing.StandardScaler(X)
#scal = preprocessing.scale(X)
#performing pca
pca = PCA(n_components=1)
pca.fit(X)
X_pca = pca.transform(X)

#add the pca value to the batting dataframe
bfns['PCA']= X_pca

In [11]:
bfns.head()

,playerID,yearID,PA,OBP,SLUG,BA,PCA
0,abreujo02,2015.0,668.0,0.347305,0.502447,0.290375,-0.136432
1,ackledu01,2015.0,130.5,0.301471,0.509719,0.251758,-0.109280
2,adamecr01,2015.0,57.0,0.298246,0.301887,0.245283,0.067423
3,adamsma01,2015.0,186.0,0.279570,0.377143,0.240000,0.014556
4,adriaeh01,2015.0,132.0,0.303030,0.265487,0.185841,0.117006


In [12]:
#convert to csv file
bfns.to_csv("batting2015.csv", index = False)

In [14]:
mf.head()

,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,...,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID
0,aardsda01,1981.0,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,...,Aardsma,David Allan,220.0,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01
1,aaronha01,1934.0,2.0,5.0,USA,AL,Mobile,NaN,NaN,NaN,...,Aaron,Henry Louis,180.0,72.0,R,R,1954-04-13,1976-10-03,aaroh101,aaronha01
2,aaronto01,1939.0,8.0,5.0,USA,AL,Mobile,1984.0,8.0,16.0,...,Aaron,Tommie Lee,190.0,75.0,R,R,1962-04-10,1971-09-26,aarot101,aaronto01
3,aasedo01,1954.0,9.0,8.0,USA,CA,Orange,NaN,NaN,NaN,...,Aase,Donald William,190.0,75.0,R,R,1977-07-26,1990-10-03,aased001,aasedo01
4,abadan01,1972.0,8.0,25.0,USA,FL,Palm Beach,NaN,NaN,NaN,...,Abad,Fausto Andres,184.0,73.0,L,L,2001-09-10,2006-04-13,abada001,abadan01


In [15]:
mf = mf.drop(['birthCountry','birthState','birthCity','deathYear','birthYear', 'birthMonth', 
              'birthDay', 'birthCountry','deathMonth','deathDay','deathCountry','deathState',
              'deathCity','weight', 'height', 'bats', 'throws'], axis = 1)

In [16]:
mf.head()

,playerID,nameFirst,nameLast,nameGiven,debut,finalGame,retroID,bbrefID
0,aardsda01,David,Aardsma,David Allan,2004-04-06,2015-08-23,aardd001,aardsda01
1,aaronha01,Hank,Aaron,Henry Louis,1954-04-13,1976-10-03,aaroh101,aaronha01
2,aaronto01,Tommie,Aaron,Tommie Lee,1962-04-10,1971-09-26,aarot101,aaronto01
3,aasedo01,Don,Aase,Donald William,1977-07-26,1990-10-03,aased001,aasedo01
4,abadan01,Andy,Abad,Fausto Andres,2001-09-10,2006-04-13,abada001,abadan01


In [17]:
mf.tail()

,playerID,nameFirst,nameLast,nameGiven,debut,finalGame,retroID,bbrefID
18841,zupofr01,Frank,Zupo,Frank Joseph,1957-07-01,1961-05-09,zupof101,zupofr01
18842,zuvelpa01,Paul,Zuvella,Paul,1982-09-04,1991-05-02,zuvep001,zuvelpa01
18843,zuverge01,George,Zuverink,George,1951-04-21,1959-06-15,zuveg101,zuverge01
18844,zwilldu01,Dutch,Zwilling,Edward Harrison,1910-08-14,1916-07-12,zwild101,zwilldu01
18845,zychto01,Tony,Zych,Anthony Aaron,2015-09-04,2015-10-03,zycht001,zychto01


In [18]:
mf['finalGame'] = pd.to_datetime(mf['finalGame'], format = '%Y-%m-%d')

In [19]:
mf = mf[(mf['finalGame'].dt.year == 2015)]

In [20]:
mf.shape

(1348, 8)

In [21]:
mf['nameID'] = mf['nameFirst'] + mf['nameLast']

In [22]:
mf.to_csv('Mastedit.csv',index = False)

In [25]:
me = pd.read_csv('Mastedit.csv')

In [26]:
me.head()

,playerID,nameFirst,nameLast,nameGiven,debut,finalGame,retroID,bbrefID,nameID
0,aardsda01,David,Aardsma,David Allan,2004-04-06,2015-08-23,aardd001,aardsda01,DavidAardsma
1,abadfe01,Fernando,Abad,Fernando Antonio,2010-07-28,2015-10-03,abadf001,abadfe01,FernandoAbad
2,abreujo02,Jose,Abreu,Jose Dariel,2014-03-31,2015-10-03,abrej003,abreujo02,JoseAbreu
3,achteaj01,A. J.,Achter,Adam Joseph,2014-09-03,2015-10-04,achta001,achteaj01,A. J.Achter
4,ackledu01,Dustin,Ackley,Dustin Michael,2011-06-17,2015-10-04,ackld001,ackledu01,DustinAckley


In [28]:
pmf = pd.merge(mf,me, how = 'left', left_on = 'playerID', right_on = 'playerID')

In [31]:
pmf.head()

,playerID,nameFirst_x,nameLast_x,nameGiven_x,debut_x,finalGame_x,retroID_x,bbrefID_x,nameID_x,nameFirst_y,nameLast_y,nameGiven_y,debut_y,finalGame_y,retroID_y,bbrefID_y,nameID_y
0,aardsda01,David,Aardsma,David Allan,2004-04-06,2015-08-23,aardd001,aardsda01,DavidAardsma,David,Aardsma,David Allan,2004-04-06,2015-08-23,aardd001,aardsda01,DavidAardsma
1,abadfe01,Fernando,Abad,Fernando Antonio,2010-07-28,2015-10-03,abadf001,abadfe01,FernandoAbad,Fernando,Abad,Fernando Antonio,2010-07-28,2015-10-03,abadf001,abadfe01,FernandoAbad
2,abreujo02,Jose,Abreu,Jose Dariel,2014-03-31,2015-10-03,abrej003,abreujo02,JoseAbreu,Jose,Abreu,Jose Dariel,2014-03-31,2015-10-03,abrej003,abreujo02,JoseAbreu
3,achteaj01,A. J.,Achter,Adam Joseph,2014-09-03,2015-10-04,achta001,achteaj01,A. J.Achter,A. J.,Achter,Adam Joseph,2014-09-03,2015-10-04,achta001,achteaj01,A. J.Achter
4,ackledu01,Dustin,Ackley,Dustin Michael,2011-06-17,2015-10-04,ackld001,ackledu01,DustinAckley,Dustin,Ackley,Dustin Michael,2011-06-17,2015-10-04,ackld001,ackledu01,DustinAckley


In [29]:
pmf = pmf.drop(columns = 'retroID', axis = 1)

KeyError: "['retroID'] not found in axis"

In [ ]:
pmf.to_csv('bat.csv',index = False)